In [0]:
import matplotlib.pyplot as plt
import numpy as np
import gzip
import pickle
import librosa

import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Bidirectional
from keras.layers import ConvLSTM2D
from keras.utils import np_utils

from keras.models import Model
from keras.layers import Dense, Input
from keras.datasets import mnist
from keras.regularizers import l1
from keras.optimizers import Adam

import pandas as pd

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
pickle_off=gzip.open("drive/My Drive/Colab Notebooks/Data/Dexed_onenote.data","rb")
emp=pickle.load(pickle_off)

In [14]:
sampling_rate=44100
nsamples=1000;
nmfccs=24;
nchroma=24;
ncontrast=6;
nbands=6;
ntrain=600;
nfeatures=131*(nmfccs)
n_y = len(emp[0][1])

X=np.zeros((nsamples,131,nmfccs+nchroma+6+ncontrast+1+1))
y=np.zeros((nsamples,n_y))
for i in range(nsamples):
    mfccTemp=librosa.feature.mfcc(y=np.array(emp[i][0]), sr=sampling_rate,n_mfcc=nmfccs)
    censTemp=librosa.feature.chroma_cens(y=np.array(emp[i][0]), sr=sampling_rate,n_chroma=nchroma)
    toneTemp=librosa.feature.tonnetz(y=np.array(emp[i][0]), sr=sampling_rate)
    spectralBandTemp=librosa.feature.spectral_bandwidth(y=np.array(emp[i][0]))
    spectralContrastTemp=librosa.feature.spectral_contrast(y=np.array(emp[i][0]),n_bands=nbands)
    spectralFlatnessTemp=librosa.feature.spectral_flatness(y=np.array(emp[i][0]))
    spectralRollTemp=librosa.feature.spectral_rolloff(y=np.array(emp[i][0]))
    feature=np.concatenate((mfccTemp,censTemp))
    feature=np.concatenate((feature,toneTemp))
    feature=np.concatenate((feature,spectralBandTemp))
    feature=np.concatenate((feature,spectralContrastTemp))
    X[i,:]=np.transpose(feature)
    y[i,:]=([emp[i][1][j][1] for j in range(0,n_y)])

/usr/local/lib/python3.6/dist-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


In [0]:
numpy.random.seed(seed=5)
trainIndex=set(np.random.choice(1000,size=ntrain,replace=False))
testIndex=set(np.linspace(0,999,1000))-trainIndex

In [0]:
x_train=X[np.array(list(trainIndex),dtype=int),:]
y_train=y[np.array(list(trainIndex),dtype=int),:]
x_test=X[np.array(list(testIndex),dtype=int),:]
y_test=y[np.array(list(testIndex),dtype=int),:]

In [0]:
def diffSizes(data, divide_by):
  ncol = data.shape[1]
  input_size = ncol
  hidden_size = ncol - ((ncol//divide_by)//2)
  code_size = ncol//divide_by
  return input_size, hidden_size, code_size

def runAEC(x_train,x_test,x_full):
  input_size, hidden_size, code_size = diffSizes(x_train, 2)
  input_img = Input(shape=(input_size,))
  hidden_1 = Dense(hidden_size, activation='relu')(input_img)
  code = Dense(code_size, activation='relu')(hidden_1)
  hidden_2 = Dense(hidden_size, activation='relu')(code)
  output_img = Dense(input_size, activation='sigmoid')(hidden_2)
  autoencoder = Model(input_img, output_img)
  autoencoder.compile(optimizer='rmsprop', loss='mse')
  autoencoder.fit(x_train, x_train, epochs=10)
  score = autoencoder.evaluate(x_test, x_test, batch_size=100)
  print(np.sqrt(score))
  #autoencoder.summary()
  encoder = Model(inputs = input_img, outputs = code)
  encoded_input = Input(shape = (code_size, ))
  encoded_train = pd.DataFrame(encoder.predict(x_full))
  encoded_train = encoded_train.add_prefix('feature_')
  return encoded_train

In [18]:
synth1_encoded = runAEC(y_train,y_test,y)
y_ae0 = np.array(synth1_encoded)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
600/600 [==============================] - 0s 346us/step - loss: 0.0832
Epoch 2/10
600/600 [==============================] - 0s 102us/step - loss: 0.0768
Epoch 3/10
600/600 [==============================] - 0s 80us/step - loss: 0.0758
Epoch 4/10
600/600 [==============================] - 0s 86us/step - loss: 0.0748
Epoch 5/10
600/600 [==============================] - 0s 76us/step - loss: 0.0738
Epoch 6/10
600/600 [==============================] - 0s 89us/step - loss: 0.0728
Epoch 7/10
600/600 [==============================] - 0s 83us/step - loss: 0.0717
Epoch 8/10
600/600 [==============================] - 0s 83us/step - loss: 0.0708
Epoch 9/10
600/600 [==============================] - 0s 80us/step - loss: 0.0696
Epoch 10/10
400/400 [==============================] - 0s 90us/step
0.26686664300202645


In [0]:

y_train_reduced=y_ae0[np.array(list(trainIndex),dtype=int),:]
y_test_reduced=y_ae0[np.array(list(testIndex),dtype=int),:]

In [20]:
model=Sequential()
model.add(LSTM(24, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(y_train_reduced.shape[1], activation='relu'))
model.compile(optimizer='Nadam',
              loss='mse')
model.fit(x_train, y_train_reduced,
          epochs=40,
          batch_size=10)
score = model.evaluate(x_test, y_test_reduced, batch_size=100)
model.save('drive/My Drive/Comp562LSTM24Nadam24.h5')
print(np.sqrt(score))
#0.2948985627415366


Epoch 1/40
600/600 [==============================] - 5s 8ms/step - loss: 0.7394
Epoch 2/40
600/600 [==============================] - 4s 6ms/step - loss: 0.4664
Epoch 3/40
600/600 [==============================] - 4s 6ms/step - loss: 0.2823
Epoch 4/40
600/600 [==============================] - 4s 7ms/step - loss: 0.1964
Epoch 5/40
600/600 [==============================] - 4s 6ms/step - loss: 0.1756
Epoch 6/40
600/600 [==============================] - 4s 6ms/step - loss: 0.1638
Epoch 7/40
600/600 [==============================] - 4s 6ms/step - loss: 0.1632
Epoch 8/40
600/600 [==============================] - 4s 6ms/step - loss: 0.1629
Epoch 9/40
600/600 [==============================] - 4s 6ms/step - loss: 0.1627
Epoch 10/40
600/600 [==============================] - 4s 6ms/step - loss: 0.1625
Epoch 11/40
600/600 [==============================] - 4s 7ms/step - loss: 0.1623
Epoch 12/40
600/600 [==============================] - 4s 7ms/step - loss: 0.1622
Epoch 13/40
600/600 [====

In [0]:
pred_y_0 = model.predict(x_test)


In [0]:
def decode(df,orig_test):
    input_size, hidden_size, code_size = diffSizes(orig_test, 2)
    input_img = Input(shape=(input_size,))
    hidden_1 = Dense(hidden_size, activation='relu')(input_img)
    encoded_input = Input(shape = (code_size, ))
    #code = Dense(code_size, activation='relu')(hidden_1)
    hidden_2 = Dense(hidden_size, activation='relu')(encoded_input)
    output_img = Dense(input_size, activation='sigmoid')(hidden_2)
    encoder = Model(inputs = encoded_input, outputs = output_img)
    decoded_test = encoder.predict(df)
    decoded_y_0 = np.array(pd.DataFrame(decoded_test))
    rmse = np.sqrt(np.mean((decoded_y_0 - orig_test)**2))
    return rmse

In [23]:
decode(pred_y_0, y_test)

0.32828756929612407

In [0]:
model=Sequential()
model.add(LSTM(24, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(255, activation='relu'))
model.compile(optimizer='RMSprop',
              loss='mse')
model.fit(x_train, y_train,
          epochs=40,
          batch_size=10)
score = model.evaluate(x_test, y_test, batch_size=100)
model.save('drive/My Drive/Comp562LSTM24RMSprop24.h5')
print(np.sqrt(score))
#0.3024906623350846


Epoch 1/40
600/600 [==============================] - 6s 10ms/step - loss: 0.1823
Epoch 2/40
600/600 [==============================] - 4s 6ms/step - loss: 0.1213
Epoch 3/40
600/600 [==============================] - 4s 7ms/step - loss: 0.1071
Epoch 4/40
600/600 [==============================] - 4s 6ms/step - loss: 0.1019
Epoch 5/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0992
Epoch 6/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0983
Epoch 7/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0976
Epoch 8/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0972
Epoch 9/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0962
Epoch 10/40
600/600 [==============================] - 4s 6ms/step - loss: 0.0950
Epoch 11/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0944
Epoch 12/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0936
Epoch 13/40
600/600 [===

In [0]:
model=Sequential()
model.add(LSTM(24, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(255, activation='relu'))
model.compile(optimizer='sgd',
              loss='mse')
model.fit(x_train, y_train,
          epochs=40,
          batch_size=10)
score = model.evaluate(x_test, y_test, batch_size=100)
model.save('drive/My Drive/Comp562LSTM24sgd24.h5')
print(np.sqrt(score))
#0.42599756100761205

Epoch 1/40
600/600 [==============================] - 6s 11ms/step - loss: 0.2668
Epoch 2/40
600/600 [==============================] - 4s 7ms/step - loss: 0.2618
Epoch 3/40
600/600 [==============================] - 4s 7ms/step - loss: 0.2590
Epoch 4/40
600/600 [==============================] - 4s 7ms/step - loss: 0.2557
Epoch 5/40
600/600 [==============================] - 4s 7ms/step - loss: 0.2525
Epoch 6/40
600/600 [==============================] - 4s 7ms/step - loss: 0.2495
Epoch 7/40
600/600 [==============================] - 4s 7ms/step - loss: 0.2468
Epoch 8/40
600/600 [==============================] - 4s 6ms/step - loss: 0.2438
Epoch 9/40
600/600 [==============================] - 4s 6ms/step - loss: 0.2412
Epoch 10/40
600/600 [==============================] - 4s 7ms/step - loss: 0.2385
Epoch 11/40
600/600 [==============================] - 4s 6ms/step - loss: 0.2357
Epoch 12/40
600/600 [==============================] - 4s 7ms/step - loss: 0.2330
Epoch 13/40
600/600 [===

In [0]:
model=Sequential()
model.add(LSTM(24, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(Dense(255, activation='relu'))
model.compile(optimizer='adagrad',
              loss='mse')
model.fit(x_train, y_train,
          epochs=40,
          batch_size=10)
score = model.evaluate(x_test, y_test, batch_size=100)
model.save('drive/My Drive/Comp562LSTM24Adagrad24.h5')
print(np.sqrt(score))
#0.30804447894382314

Epoch 1/40
600/600 [==============================] - 7s 11ms/step - loss: 0.1345
Epoch 2/40
600/600 [==============================] - 4s 7ms/step - loss: 0.1055
Epoch 3/40
600/600 [==============================] - 4s 7ms/step - loss: 0.1022
Epoch 4/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0997
Epoch 5/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0971
Epoch 6/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0966
Epoch 7/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0963
Epoch 8/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0961
Epoch 9/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0964
Epoch 10/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0970
Epoch 11/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0967
Epoch 12/40
600/600 [==============================] - 4s 7ms/step - loss: 0.0945
Epoch 13/40
600/600 [===

In [0]:
model2=Sequential()
model2.add(Bidirectional(LSTM(24, input_shape=(x_train.shape[1], x_train.shape[2]))))
model2.add(Dense(255, activation='relu'))
model2.compile(optimizer='Nadam',
              loss='mse')
model2.fit(x_train, y_train,
          epochs=40,
          batch_size=10)
score = model2.evaluate(x_test, y_test, batch_size=100)
model2.save('drive/My Drive/Comp562/BiLSTM24Nadam24.h5')
print(np.sqrt(score))
#0.3131173651729559

Epoch 1/40
600/600 [==============================] - 9s 16ms/step - loss: 0.1444
Epoch 2/40
600/600 [==============================] - 6s 10ms/step - loss: 0.1025
Epoch 3/40
600/600 [==============================] - 6s 10ms/step - loss: 0.0999
Epoch 4/40
600/600 [==============================] - 6s 10ms/step - loss: 0.0990
Epoch 5/40
600/600 [==============================] - 6s 10ms/step - loss: 0.0985
Epoch 6/40
600/600 [==============================] - 6s 10ms/step - loss: 0.0978
Epoch 7/40
600/600 [==============================] - 6s 10ms/step - loss: 0.0970
Epoch 8/40
600/600 [==============================] - 6s 10ms/step - loss: 0.0966
Epoch 9/40
600/600 [==============================] - 6s 10ms/step - loss: 0.0964
Epoch 10/40
600/600 [==============================] - 6s 10ms/step - loss: 0.0962
Epoch 11/40
600/600 [==============================] - 6s 10ms/step - loss: 0.0960
Epoch 12/40
600/600 [==============================] - 6s 10ms/step - loss: 0.0960
Epoch 13/40
6

In [0]:
model1=Sequential()
model1.add(GRU(24, input_shape=(x_train.shape[1], x_train.shape[2])))
model1.add(Dense(255, activation='relu'))
model1.compile(optimizer='Nadam',
              loss='mse')
model1.fit(x_train, y_train,
          epochs=40,
          batch_size=10)
score = model1.evaluate(x_test, y_test, batch_size=100)
model1.save('drive/My Drive/Comp562/GRU24Nadam24.h5')
print(np.sqrt(score))
#0.30473644218082063

Epoch 1/40
600/600 [==============================] - 6s 11ms/step - loss: 0.1525
Epoch 2/40
600/600 [==============================] - 3s 6ms/step - loss: 0.1019
Epoch 3/40
600/600 [==============================] - 3s 6ms/step - loss: 0.0957
Epoch 4/40
600/600 [==============================] - 3s 6ms/step - loss: 0.0930
Epoch 5/40
600/600 [==============================] - 3s 6ms/step - loss: 0.0924
Epoch 6/40
600/600 [==============================] - 3s 5ms/step - loss: 0.0919
Epoch 7/40
600/600 [==============================] - 3s 5ms/step - loss: 0.0915
Epoch 8/40
600/600 [==============================] - 3s 6ms/step - loss: 0.0913
Epoch 9/40
600/600 [==============================] - 3s 5ms/step - loss: 0.0912
Epoch 10/40
600/600 [==============================] - 3s 5ms/step - loss: 0.0910
Epoch 11/40
600/600 [==============================] - 3s 5ms/step - loss: 0.0910
Epoch 12/40
600/600 [==============================] - 3s 5ms/step - loss: 0.0907
Epoch 13/40
600/600 [===